In [1]:
%cd /Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions/

/Users/natalipeeva/Documents/GitHub/Automatic-Answering-of-City-Council-Questions


In [2]:
import sys
import os
src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(src_dir)

from read_data.read_data import read_urls_questions, get_questions, get_url_content_tuples, get_relevant_docs
from elasticsearch import Elasticsearch
from retrieval.sparse_retrieval.bm25 import set_index, get_result_tuples

In [3]:
import pandas as pd

In [4]:
collected, questions =  read_urls_questions(os.path.join('data/reference_urls/reference_urls_collected.csv'),
                                            os.path.join('data/question_answer/questions_updated_urls.csv'), clean_url_nan=True) # read collected urls and questions + remove unsuccessful collection

In [5]:
amsterdam_only = questions[questions['Cleaned_URLs'].apply(lambda urls: any(url.startswith('https://www.amsterdam.nl') for url in urls))]

In [6]:
amsterdam_collected = pd.read_csv('data/collected/combined.csv')

In [7]:
len(amsterdam_collected)

8851

In [8]:
amsterdam_references = collected[collected['URL'].str.startswith('https://www.amsterdam.nl')]

In [9]:
amsterdam_missing = pd.DataFrame()  # Empty DataFrame to store missing rows
missing_rows = []

for index, row in amsterdam_references.iterrows():
    if row['URL'] not in amsterdam_collected['URL'].tolist():
        missing_rows.append(row.to_dict())

amsterdam_missing = pd.DataFrame(missing_rows)


In [10]:
all_amsterdam = pd.concat([amsterdam_collected, amsterdam_missing])

In [11]:
len(all_amsterdam)

8908

In [12]:
es_client = Elasticsearch("http://localhost:9200")

In [13]:
question_list = get_questions(amsterdam_only)
document_list = get_url_content_tuples(all_amsterdam)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [14]:
es_client = set_index(es_client=es_client, collected=all_amsterdam, mappings=mappings) # 48 secs

In [15]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 15 gives the best results ; when i set it to 5 gives 0,26

In [18]:
results[list(results.keys())[0]][1]

TypeError: list indices must be integers or slices, not str

In [72]:
from irmetrics.topk import recall

In [73]:
predictions = []
for question in results.keys():
    urls = []
    for result in results[question][1]:
        urls.append(result['url'])
    predictions.append(list(set(urls)))

relevant_docs = get_relevant_docs(amsterdam_only)

In [46]:
import numpy as np
from irmetrics.topk import rr

In [74]:
true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =7)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


Average Mean Reciprocal Rank: 0.044651011089367254


In [40]:
from irmetrics.topk import recall

In [76]:
# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_recall)

Average Mean Reciprocal Rank: 0.1232876712328767


### Passages

In [12]:
import pandas as pd
from nltk.tokenize import word_tokenize
import nltk

# Download NLTK resources for Dutch tokenization
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /Users/natalipeeva/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

In [13]:
# Create an empty dataframe for the new data
new_df = pd.DataFrame(columns=['URL', 'Textual_Content'])

# Iterate over each row in the original dataframe
for index, row in all_amsterdam.iterrows():
    url = row['URL']
    content = str(row['Textual_Content'])
    
    # Tokenize the content into words
    words = word_tokenize(content, language='dutch')
    
    # Create passages of 100 words
    passages = [' '.join(words[i:i+100]) for i in range(0, len(words), 100)]
    
    # Create a new dataframe for the passages
    passage_df = pd.DataFrame({'URL': url, 'Textual_Content': passages})
    
    # Concatenate the new dataframe with the main dataframe
    new_df = pd.concat([new_df, passage_df], ignore_index=True)


In [14]:
new_df.to_csv('data/amsterdam/amsterdam_passages.csv')

In [129]:
document_list = get_url_content_tuples(new_df)
mappings = {
        "properties": {
            "url": {"type": "text"},
            "text": {"type": "text", "analyzer": "standard", "similarity": "BM25"}
    }
}

In [130]:
es_client = set_index(es_client=es_client, collected=new_df, mappings=mappings) # 7secs

In [148]:
results = get_result_tuples(es_client=es_client, questions=question_list, n=10) # 1:45 min

In [149]:
predictions = []
for question in results.keys():
    urls = []
    for result in results[question][1]:
        urls.append(result['url'])
    predictions.append(list(set(urls)))

relevant_docs = get_relevant_docs(amsterdam_only)

true = []
for question in relevant_docs.keys():
    true.append(list(set(relevant_docs[question])))

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = rr(true_values, predictions[i], k =1000)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_mrr = np.mean(mrr_values)

print("Average Mean Reciprocal Rank:", average_mrr)


from irmetrics.topk import recall

# Calculate the Mean Reciprocal Rank for each question
mrr_values = []
for i in range(len(predictions)):
    true_values = true[i]
    mrr = recall(true_values, predictions[i], k=10)
    mrr_values.append(mrr)

# Calculate the average Mean Reciprocal Rank
average_recall = np.mean(mrr_values)

print("Average Recall:", average_recall)

Average Mean Reciprocal Rank: 0.0319634703196347
Average Recall: 0.0684931506849315


### Save Data

In [78]:
amsterdam_only.to_csv('data/amsterdam/only_amsterdam_questions.csv')

In [79]:
import pickle
with open ('data/amsterdam/bm25_documents_amsterdam.pickle', 'wb') as f:
    pickle.dump(results, f)

In [127]:
all_amsterdam.to_csv('data/amsterdam/amsterdam_full.csv')

In [150]:
with open ('data/amsterdam/bm25_passages10_amsterdam.pickle', 'wb') as f:
    pickle.dump(results, f)

# more results: 
- if it's top 500: Average Recall: 0.3561643835616438; Average Mean Reciprocal Rank: 0.010190664748755714
- If it's 200: 0,28
- If 100: 0,21
- 1000: Average Mean Reciprocal Rank: 0.008540249382361066; 0.3972602739726027
- If 150: Average Mean Reciprocal Rank: 0.019564395660491188 ; Average Recall: 0.2671232876712329
- If 10: